**Imports**

In [ ]:
# UMAP 2.0.0: Updated December 2022 (HDC) 
# Adapted from JACS, 2021, 143, 19078-19090; https://xinhaoli74.github.io/posts/2019/09/interactive-map/
import os,re,sys,pickle,datetime,time,random,itertools
import warnings
warnings.filterwarnings("ignore")
import numpy as np
np.set_printoptions(threshold=sys.maxsize)
import openpyxl
from openpyxl import load_workbook
import pandas as pd
import math
import umap
import umap.plot
from bokeh.plotting import figure, show, save, output_notebook, output_file
from bokeh.resources import INLINE 
from bokeh.models import HoverTool, ColumnDataSource, CategoricalColorMapper
from bokeh.transform import factor_cmap
from bokeh.models import Legend, LegendItem
from bokeh.palettes import RdBu3,RdBu9,Spectral6
from sklearn import metrics
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler,MinMaxScaler,PolynomialFeatures
import sklearn.cluster as cluster
from IPython.display import SVG
from IPython.display import HTML
import matplotlib.pyplot as plt
import hdbscan
randomstate = 42
import rdkit
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit import RDConfig
from rdkit.Chem import MolFromSmiles
from rdkit.Chem import PandasTools
from rdkit.Chem import Descriptors
from rdkit.ML.Descriptors import MoleculeDescriptors
from rdkit.Chem.Draw import MolsToGridImage
from rdkit.Chem import PropertyMol
from rdkit.Chem import Draw
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem.Draw import rdMolDraw2D
from rdkit.Chem.Draw.MolDrawing import Font
from rdkit.Chem import rdmolfiles
from rdkit.Chem import rdFMCS
from rdkit.Chem.PandasTools import ChangeMoleculeRendering
from rdkit.Chem import rdDepictor
from rdkit.Chem.Draw.MolDrawing import MolDrawing,DrawingOptions
DrawingOptions.bondLineWidth=1.8
DrawingOptions.includeAtomNumbers=False
size = (100, 100)

**Load data**

In [ ]:
### Manual Input: #################
excel_file ="QikProp_UMAP_input" 
excel_sheet = "Sheet1" 
nsamp = 1507
num_par = 51  
par_start_col = 4 # 0-index
smiles_col = 3 # 0-index
xlabelrow = True
##################################
# Feature DF
inp = pd.read_excel(excel_file+".xlsx",excel_sheet,index_col=0,nrows=nsamp+int(xlabelrow),header=1, usecols=list(range(0,(num_par+par_start_col))),engine='openpyxl')
feats = inp[inp.columns[(smiles_col):]].loc[inp.index[:]]
feats.index = feats.index.astype(int)
# Smiles DF
smiles = pd.read_excel(excel_file+".xlsx",excel_sheet,header=1,index_col=0,usecols=list(range(0,(smiles_col+1))),engine='openpyxl')
# Feature Scaling
X_all = np.array(feats)
scaler = StandardScaler()
scaler.fit(X_all)
X_all_sc = scaler.transform(X_all)

**Remove colinear descriptors**

In [ ]:
threshold = 0.95
print(f'Shape of descriptors file before removing parameters with R^2 > {threshold} :',feats.shape)
df_corr = feats.corr()
df_not_correlated = ~(df_corr.mask(np.tril(np.ones([len(df_corr)]*2, dtype=bool))).abs() > threshold).any()
un_corr_idx = df_not_correlated.loc[df_not_correlated[df_not_correlated.index] == True].index
feats = feats[un_corr_idx]
print(f'Shape of descriptors file after removing parameters with R^2 > {threshold} : ',feats.shape)

**Convert SMILES to strucutres with RDkit and prepare settings for interactive map strucutres**

In [ ]:
# Creates structure from SMILES column. 
# IF THIS CELL GIVES ERRORS, FIRST CHECK YOUR SMILES STRINGS!!
# The df is embedded as an HTML object because of version compatibility between pandas and RDKit
def show_df(df):
    return HTML(df.to_html(notebook=True))
smiles_df = pd.DataFrame({'name':smiles.name[:nsamp],
                          'SMILES':smiles.SMILES[:nsamp],
                          'type':smiles.type[:nsamp],})
PandasTools.AddMoleculeColumnToFrame(smiles_df,'SMILES')
show_df(smiles_df)

In [ ]:
### Create structures in the interactive plot ##########
def _prepareMol(mol,kekulize):
    mc = Chem.Mol(mol.ToBinary())
    if kekulize:
        try:
            Chem.Kekulize(mc)
        except:
            mc = Chem.Mol(mol.ToBinary())
    if not mc.GetNumConformers():
        rdDepictor.Compute2DCoords(mc)
    return mc

def moltosvg(mol,molSize=(175,90),kekulize=True,drawer=None,**kwargs):
    mc = _prepareMol(mol,kekulize)
    if drawer is None:
        drawer = rdMolDraw2D.MolDraw2DSVG(molSize[0],molSize[0])
    drawer.DrawMolecule(mc,**kwargs)
    drawer.FinishDrawing()
    svg = drawer.GetDrawingText()
    return SVG(svg.replace('svg:',''))

svgs = [moltosvg(m).data for m in smiles_df.ROMol]

**Build the UMAP space and interactive plot with structures**

In [ ]:
### This cell builds the UMAP and prints an interactive view, first WITHOUT structures.
### Update "hover_data" as necessary.
hover_data = pd.DataFrame({'index':smiles_df.index[:nsamp],
                           'name':smiles_df.name[:nsamp],
                           'type':smiles_df.type[:nsamp]
                          })

# print(hover_data)
reducer = umap.UMAP(random_state=2)
reduced = reducer.fit(X_all_sc)
X_all_red = reducer.transform(X_all_sc)

umap.plot.output_notebook()
p=umap.plot.interactive(reduced, labels=smiles.type, hover_data=hover_data, point_size=5, alpha=0.7)
reduced_labeled = pd.DataFrame({'x':X_all_red[:nsamp,0],
                                'y':X_all_red[:nsamp,1],
                                'index':smiles_df.index[:nsamp],
                                'name':smiles_df.name[:nsamp],
                                'type':smiles_df.type[:nsamp]
                          })


umap.plot.show(p)

In [ ]:
# Make an interactive HTML plot (with hover structures) from the above chemical space; colors based on molecule 'type'
title='UMAP'
file_name = 'InteractiveMap_subsets-FINAL'

source = ColumnDataSource(data=dict(x=X_all_red[:,0], y=X_all_red[:,1], desc=smiles_df.name,svgs=svgs[:],type=smiles_df.type[:]))
types = smiles_df['type'].unique()
mapper=factor_cmap('type', palette=Spectral6, factors=types)
ChangeMoleculeRendering(renderer='PNG')
hover = HoverTool(tooltips="""
    <div>
        <div>@svgs{safe}
        </div>
        <div>
            <span style="font-size: 14px; font-weight: bold;">@desc</span>
        </div>
    </div>
    """
)

interactive_map = figure(plot_width=500, plot_height=500, tools=['reset,box_zoom,wheel_zoom,zoom_in,zoom_out,pan',hover],
           title=title,x_axis_label="Dimension 1",y_axis_label="Dimension 2")
interactive_map.circle('x', 'y', size=10, source=source, fill_alpha=0.7,fill_color=mapper,line_color='white')

########################################
### Change legend here ####
########################################
### SVGs #############
source0 = ColumnDataSource(data=dict(x=X_all_red[0:1507,0], y=X_all_red[0:1507,1],svgs=svgs[0:1507]))
### FDA ################
source1 = ColumnDataSource(data=dict(x=X_all_red[0:1441,0], y=X_all_red[0:1441,1], desc=smiles_df.name[0:1441],svgs=svgs[0:1441]))
### Metabolites #######
source2 = ColumnDataSource(data=dict(x=X_all_red[1442:1507,0], y=X_all_red[1442:1507,1], desc=smiles_df.name[1442:1507],svgs=svgs[1442:1507]))

interactive_map.circle('x', 'y', size=10, source=source1, fill_alpha=0.8,fill_color='#0C3B8C',line_color='white',legend_label="FDA");
interactive_map.circle('x', 'y', size=10, source=source2, fill_alpha=0.8,fill_color='#BE1111',line_color='white',legend_label="Metabolite");
# interactive_map.circle('x', 'y', size=10, source=source0, fill_alpha=0.0,line_width=0); # change line width to show the main set 
interactive_map.legend.location = "top_left"
interactive_map.legend.title = "Molecule Groups"

show(interactive_map)
# output_file(f"{file_name}.html")
# save(interactive_map)

In [ ]:
# UMAP Embedding for HDBSAN clustering- first make clusterable embedding of UMAP (https://umap-learn.readthedocs.io/en/latest/clustering.html)
clusterable_embedding = umap.UMAP(
    n_neighbors=50,
    min_dist=0.0,
    n_components=2,
    random_state=42,
).fit_transform(X_all_sc.data)
plt.scatter(clusterable_embedding[:, 0], clusterable_embedding[:, 1], s=3, cmap='Spectral');

In [ ]:
# HDBSAN clustering on above embedding
labels = hdbscan.HDBSCAN(
    min_samples=5,
    min_cluster_size=30,
    ).fit_predict(clusterable_embedding)
clustered = (labels >= 0)
plt.figure(figsize=(8,8)) 
plt.scatter(X_all_red[~clustered, 0],
            X_all_red[~clustered, 1],
            color=(0.5, 0.5, 0.5),
            s=10,
            alpha=0.5)
plt.scatter(X_all_red[clustered, 0],
            X_all_red[clustered, 1],
            c=labels[clustered],
            s=10,
            cmap='Spectral');
# Fraction of points clustered:
print("Fraction of points clustered:",np.sum(clustered) / X_all_red.data.shape[0])
HDBSCAN_clusters = pd.DataFrame({'x':X_all_red[:nsamp,0],
                                'y':X_all_red[:nsamp,1],
                                'index':smiles_df.index[:nsamp],
                                'name':smiles_df.name[:nsamp],
                                'type':smiles_df.type[:nsamp],
                                'cluster':labels[:nsamp]
                          })
# print(reduced_HD_clusters)
HDBSCAN_clusters.to_excel(r'HDBSCAN_clusters.xlsx', index = False)

In [ ]:
# Interactive Map with Clusters
# Make an interactive HTML plot with colors based clusters
from bokeh.transform import linear_cmap
from bokeh.models import Legend, LegendItem
from bokeh.palettes import Viridis11
title='UMAP with Clusters'
file_name1 = 'InteractiveMap_with_clustering-FINAL'

m1 = labels
mapper = linear_cmap(field_name='m1', palette=Viridis11 ,low=min(m1) ,high=max(m1))
source1 = ColumnDataSource(data=dict(x=X_all_red[:,0], y=X_all_red[:,1],m1=m1,desc= smiles_df.name, svgs=svgs))

ChangeMoleculeRendering(renderer='PNG')
hover = HoverTool(tooltips="""
    <div>
        <div>@svgs{safe}
        </div>
        <div>
            <span style="font-size: 16px; font-weight: bold;">@desc</span>
        </div>
    </div>
    """
)

interactive_map = figure(plot_width=800, plot_height=800, tools=['reset,box_zoom,wheel_zoom,zoom_in,zoom_out,pan',hover],
           x_axis_label="Dimension 1",y_axis_label="Dimension 2")
interactive_map.circle('x', 'y', size=22, source=source1, fill_alpha=0.8,fill_color=mapper,line_color='white', line_width=2)

interactive_map.xaxis.axis_label_text_font_size = "22pt"
interactive_map.yaxis.axis_label_text_font_size = "22pt"
interactive_map.yaxis.major_label_text_font_size = "22pt"
interactive_map.xaxis.major_label_text_font_size = "22pt"
interactive_map.xgrid.visible = False
interactive_map.ygrid.visible = False
show(interactive_map)
# output_file(f"{file_name1}.html")
# save(interactive_map)